(Baseline_FDS_RealWorldData)=
# Datos del mundo real 


La sección anterior ilustró cómo se pueden utilizar técnicas simples de preprocesamiento de datos y aprendizaje supervisado para diseñar un sistema de detección de fraude de referencia. Los resultados presentados se basaron en datos reproducibles, pero simulados. Apliquemos ahora exactamente la misma metodología con datos de transacciones del mundo real. Debido a razones de confidencialidad, los datos utilizados en esta sección no se pueden compartir. Si bien los resultados presentados en esta sección no se pueden reproducir, proporcionan información sobre los rendimientos que se obtendrían en un entorno del mundo real. 

El conjunto de datos utilizado fue proporcionado por Worldline, y es de naturaleza similar a los conjuntos de datos utilizados en las publicaciones referenciadas en nuestra [página de ResearchGate - Colaboración conjunta: MLG ULB y Worldline](https://www.researchgate.net/project/Fraud-detection-with-machine-learning).

## Conjuntos de entrenamiento y prueba

Más específicamente, nuestros datos del mundo real son transacciones de comercio electrónico para Bélgica de 2018. El número de transacciones diarias es de alrededor de 400.000. El número de transacciones fraudulentas diarias fue de alrededor de 1.000, dando una proporción de transacciones fraudulentas de alrededor del 0,25%. El número de tarjetas comprometidas diarias fue ligeramente superior a 300: cada tarjeta comprometida causó un promedio de 3 transacciones fraudulentas por día. La Fig. 1 resume estas estadísticas diariamente para el período 2018-07-25 a 2018-08-14.   



![alt text](images/WL_summary_stats.png)
<p style="text-align: center;">
Fig. 1. Datos de transacciones del mundo real. Número de transacciones por día (/100), transacciones fraudulentas por día y tarjetas comprometidas por día. Período: 2018-07-25 a 2018-08-14.
</p>




## Entrenamiento del modelo: Árbol de decisión

Primero entrenamos un árbol de decisión de profundidad dos. El árbol resultante se muestra en la Fig. 2. Vale la pena señalar que las reglas de detección de fraude descubiertas por el árbol de decisión difieren de las obtenidas en datos simulados. El principal patrón de fraude descubierto por el árbol (hoja azul en la parte inferior derecha) consiste en transacciones que ocurrieron en una terminal recientemente comprometida (`TERMINAL_ID_RISK_30DAY_WINDOW`>0.075), y para las cuales el cliente había realizado un gran número de transacciones en las últimas 24 horas (`CUSTOMER_ID_NB_TX_1DAY_WINDOW`>13.5). Esta regla en realidad se relaciona con un comportamiento de fraude común: Los estafadores suelen intentar realizar tantas transacciones como puedan cuando logran comprometer una tarjeta de crédito, aumentando así el número de transacciones realizadas en el último día. 

![alt text](images/WL_DecisionTree.png)
<p style="text-align: center;">
Fig. 2. Árbol de decisión de profundidad dos, obtenido utilizando una semana de datos del mundo real.
</p>


(Chapter3_BaselineWL)=
## Rendimiento utilizando modelos de predicción estándar

Luego ejecutamos los mismos cinco algoritmos de entrenamiento que en la sección anterior: regresión logística, árboles de decisión de profundidad dos y profundidad ilimitada, bosque aleatorio y XGBoost. Los resultados de rendimiento en el conjunto de prueba se informan en la Fig. 3. 

![alt text](images/WL_benchmark_test_set.png)
<p style="text-align: center;">
Fig. 3. Rendimiento de detección de fraude obtenido con cinco algoritmos de aprendizaje automático estándar en el conjunto de prueba.
</p>


El AUC ROC de todos los modelos es superior a 0.5, lo que significa que todos funcionan mejor que un clasificador aleatorio. Para las tres métricas de rendimiento, el peor modelo es el árbol de decisión con profundidad ilimitada (AUC ROC cercano a 0.5, lo que significa que no funciona mucho mejor que un clasificador aleatorio). Los mejores modelos son XGBoost y regresión logística, dependiendo de qué métrica de rendimiento se elija. Para AUC ROC y AP, XGBoost proporciona los mejores rendimientos. Para CP@100, el mejor rendimiento lo proporciona el modelo de regresión logística. 

Hay que tener en cuenta que estos resultados son preliminares y solo reflejan los rendimientos obtenidos en un pequeño subconjunto de los datos, sin ningún ajuste de los parámetros del modelo. Aún así, es notable que las tasas de detección de estos modelos de referencia, en particular para XGBoost y regresión logística, estén muy por encima de las de un clasificador aleatorio. Recordando que la proporción de transacciones fraudulentas es solo del 0.25%, la card precision@100 de un clasificador aleatorio debería ser de alrededor de 0.0025 (es decir, menos de una tarjeta comprometida detectada por día). Un modelo de regresión logística simple aumenta la CP@100 a 0.15, lo que significa que, en promedio, este clasificador permite detectar correctamente, cada día, 15 tarjetas comprometidas de las 100 tarjetas más sospechosas identificadas por el modelo de predicción. 

Además de los rendimientos de predicción en el conjunto de prueba, también calculamos los rendimientos de predicción en el conjunto de entrenamiento. Se informan en la Fig. 4. Vale la pena señalar que dos modelos proporcionan predicciones perfectas: el árbol de decisión con profundidad ilimitada y el bosque aleatorio (AUC ROC de 1). Estos resultados también se observaron para los datos sintéticos (ver sección anterior, {ref}`Baseline_FDS_Performances_Simulation`), y reflejan el fenómeno de *sobreajuste*. Los modelos menos sensibles al sobreajuste son el modelo de regresión logística y el árbol de decisión con profundidad 2. Tiene sentido que las métricas de entrenamiento de modelos basados en árboles con profundidad ilimitada (Bosque Aleatorio, Árbol de Decisión) logren una detección perfecta porque, durante el entrenamiento, estos modelos dividirán los datos recursivamente hasta la obtención de hojas puras. 

![alt text](images/WL_benchmark_train_set.png)
<p style="text-align: center;">
Fig. 4. Rendimiento de detección de fraude obtenido con cinco algoritmos de aprendizaje automático estándar en el conjunto de entrenamiento.
</p>


Finalmente informamos los tiempos de ejecución, para entrenamiento y predicciones, de estos cinco modelos de predicción. Los resultados se presentan en la Fig. 5. Vale la pena señalar que los tiempos de ejecución son mucho más altos que con el conjunto de datos simulado. En particular, los modelos de bosque aleatorio y XGBoost se ejecutaron utilizando un servidor con 20 núcleos. Sus tiempos de ejecución de entrenamiento son, por lo tanto, cerca de 100 veces más largos que la regresión logística. XGBoost también tiene una implementación optimizada para GPU que aún podría acelerar el entrenamiento en un orden de magnitud.

![alt text](images/WL_benchmark_exec_times.png)
<p style="text-align: center;">
Fig. 5. Tiempos de ejecución para el entrenamiento y las predicciones de los cinco algoritmos de aprendizaje automático.
</p>
